# Radiomics Service Client Example

This Jupyter Notebook demonstrates how to connect to a running radiomics service.

In [ ]:
import numpy as np
import SimpleITK as sitk

from pathlib import Path

from platipy.dicom.download.tcia import get_collections, get_patients_in_collection, fetch_data
from platipy.dicom.dicom_directory_crawler.conversion_utils import process_dicom_directory
from platipy.backend.client import PlatiPyClient

In [ ]:
client = PlatiPyClient("127.0.0.1", 8000, "c4df42d4-b7f4-48ba-aaea-921452ceb5a6", "PyRadiomics Extractor")

In [ ]:

collections = get_collections()
collection = "LCTSC"
collection in collections

patients = get_patients_in_collection(collection)
patient_ids = patients[0:2]

dicom_data = fetch_data(collection, patient_ids=patient_ids, output_directory="./data")


In [ ]:
data_path = Path(f"data/LCTSC")

dataset = client.add_dataset()

for patient_path in data_path.glob("*"):
    ct_image = next(patient_path.glob("NIFTI/*/IMAGES/*"))
    structures = patient_path.glob("NIFTI/*/STRUCTURES/*")

    img_obj = client.add_data_object(dataset, file_path=str(ct_image), meta_data={"patid": patient_path.name})
    for structure in structures:
        s = client.add_data_object(dataset, parent=img_obj, file_path=str(structure))

In [ ]:
settings = client.get_default_settings()
settings["radiomics"] = {"firstorder": ["10Percentile","90Percentile","Energy","Entropy","InterquartileRange","Kurtosis","Maximum"],
"shape": ["VoxelVolume", "SurfaceArea"],
"custom": ["25Percentile"]}

In [ ]:
for i in client.run_algorithm(dataset, config=settings):
    print(i)

In [ ]:
client.download_output_objects(dataset)